In [1]:
loans = read.csv("loans.csv")

In [2]:
str(loans)

'data.frame':	9578 obs. of  14 variables:
 $ credit.policy    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ purpose          : Factor w/ 7 levels "all_other","credit_card",..: 3 2 3 3 2 2 3 1 5 3 ...
 $ int.rate         : num  0.119 0.107 0.136 0.101 0.143 ...
 $ installment      : num  829 228 367 162 103 ...
 $ log.annual.inc   : num  11.4 11.1 10.4 11.4 11.3 ...
 $ dti              : num  19.5 14.3 11.6 8.1 15 ...
 $ fico             : int  737 707 682 712 667 727 667 722 682 707 ...
 $ days.with.cr.line: num  5640 2760 4710 2700 4066 ...
 $ revol.bal        : int  28854 33623 3511 33667 4740 50807 3839 24220 69909 5630 ...
 $ revol.util       : num  52.1 76.7 25.6 73.2 39.5 51 76.8 68.6 51.1 23 ...
 $ inq.last.6mths   : int  0 0 1 1 0 0 0 0 1 1 ...
 $ delinq.2yrs      : int  0 0 0 0 1 0 0 0 0 0 ...
 $ pub.rec          : int  0 0 0 0 0 0 1 0 0 0 ...
 $ not.fully.paid   : int  0 0 0 0 0 0 1 1 0 0 ...


In [3]:
summary(loans)

 credit.policy                 purpose        int.rate       installment    
 Min.   :0.000   all_other         :2331   Min.   :0.0600   Min.   : 15.67  
 1st Qu.:1.000   credit_card       :1262   1st Qu.:0.1039   1st Qu.:163.77  
 Median :1.000   debt_consolidation:3957   Median :0.1221   Median :268.95  
 Mean   :0.805   educational       : 343   Mean   :0.1226   Mean   :319.09  
 3rd Qu.:1.000   home_improvement  : 629   3rd Qu.:0.1407   3rd Qu.:432.76  
 Max.   :1.000   major_purchase    : 437   Max.   :0.2164   Max.   :940.14  
                 small_business    : 619                                    
 log.annual.inc        dti              fico       days.with.cr.line
 Min.   : 7.548   Min.   : 0.000   Min.   :612.0   Min.   :  179    
 1st Qu.:10.558   1st Qu.: 7.213   1st Qu.:682.0   1st Qu.: 2820    
 Median :10.928   Median :12.665   Median :707.0   Median : 4140    
 Mean   :10.932   Mean   :12.607   Mean   :710.8   Mean   : 4562    
 3rd Qu.:11.290   3rd Qu.:17.950   3rd 

In [4]:
table(loans$not.fully.paid)


   0    1 
8045 1533 

In [5]:
1533/(1533+8045)

[1] 0.1600543

In [6]:
missing = subset(loans, is.na(log.annual.inc) | is.na(days.with.cr.line) | is.na(revol.util) | is.na(inq.last.6mths) | is.na(delinq.2yrs) | is.na(pub.rec))

In [9]:
nrow(missing)

[1] 62

In [8]:
library(mice)

Loading required package: lattice


Attaching package: 'mice'


The following objects are masked from 'package:base':

    cbind, rbind




In [10]:
set.seed(144)

In [11]:
vars.for.imputation = setdiff(names(loans), "not.fully.paid")

In [12]:
imputed = complete(mice(loans[vars.for.imputation]))


 iter imp variable
  1   1  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   2  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   3  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   4  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  1   5  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   1  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   2  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   3  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   4  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  2   5  log.annual.inc  days.with.cr.line  revol.util  inq.last.6mths  delinq.2yrs  pub.rec
  3   1  log.annual.inc  days.with.cr.line  revol.

In [13]:
loans[vars.for.imputation] = imputed

In [14]:
library(caTools)

In [15]:
set.seed(144)

spl = sample.split(loans$not.fully.paid, 0.7)

train = subset(loans, spl == TRUE)

test = subset(loans, spl == FALSE)

In [16]:
mod = glm(not.fully.paid~., data=train, family="binomial")

In [17]:
summary(mod)


Call:
glm(formula = not.fully.paid ~ ., family = "binomial", data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2008  -0.6213  -0.4953  -0.3609   2.6389  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                9.250e+00  1.552e+00   5.959 2.54e-09 ***
credit.policy             -3.417e-01  1.009e-01  -3.388 0.000704 ***
purposecredit_card        -6.124e-01  1.344e-01  -4.557 5.18e-06 ***
purposedebt_consolidation -3.199e-01  9.179e-02  -3.485 0.000493 ***
purposeeducational         1.351e-01  1.753e-01   0.771 0.440814    
purposehome_improvement    1.728e-01  1.479e-01   1.168 0.242901    
purposemajor_purchase     -4.828e-01  2.008e-01  -2.404 0.016215 *  
purposesmall_business      4.123e-01  1.418e-01   2.907 0.003653 ** 
int.rate                   6.434e-01  2.085e+00   0.309 0.757592    
installment                1.274e-03  2.092e-04   6.091 1.12e-09 ***
log.annual.inc            -4.328e-01

In [18]:
test$predicted.risk = predict(mod, newdata=test, type="response")

In [19]:
table(test$not.fully.paid, test$predicted.risk > 0.5)

   
    FALSE TRUE
  0  2400   13
  1   457    3

In [20]:
2413/2873

[1] 0.8398886

In [21]:
library(ROCR)

pred = prediction(test$predicted.risk, test$not.fully.paid)

as.numeric(performance(pred, "auc")@y.values)

Loading required package: gplots


Attaching package: 'gplots'


The following object is masked from 'package:stats':

    lowess




[1] 0.672104

In [22]:
bivariate = glm(not.fully.paid~int.rate, data=train, family="binomial")

summary(bivariate)


Call:
glm(formula = not.fully.paid ~ int.rate, family = "binomial", 
    data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.0547  -0.6271  -0.5442  -0.4361   2.2914  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -3.6726     0.1688  -21.76   <2e-16 ***
int.rate     15.9214     1.2702   12.54   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 5896.6  on 6704  degrees of freedom
Residual deviance: 5734.8  on 6703  degrees of freedom
AIC: 5738.8

Number of Fisher Scoring iterations: 4


In [23]:
pred.bivariate = predict(bivariate, newdata=test, type="response")

summary(pred.bivariate)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.06196 0.11549 0.15077 0.15963 0.18928 0.42662 

In [24]:
prediction.bivariate = prediction(pred.bivariate, test$not.fully.paid)

as.numeric(performance(prediction.bivariate, "auc")@y.values)

[1] 0.6239081

In [25]:
10*exp(0.06*3)

[1] 11.97217

In [26]:
test$profit = exp(test$int.rate*3) - 1

test$profit[test$not.fully.paid == 1] = -1

In [27]:
summary(test$profit)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.0000  0.2858  0.4111  0.2094  0.4980  0.8895 

In [28]:
highInterest = subset(test, int.rate >= 0.15)

summary(highInterest$profit)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.0000 -1.0000  0.5992  0.2251  0.6380  0.8895 

In [29]:
table(highInterest$not.fully.paid)


  0   1 
327 110 

In [30]:
cutoff = sort(highInterest$predicted.risk, decreasing=FALSE)[100]

In [31]:
selectedLoans = subset(highInterest, predicted.risk <= cutoff) 

In [34]:
nrow(selectedLoans)

[1] 100

In [35]:
sum(selectedLoans$profit)

[1] 31.27825

In [36]:
table(selectedLoans$not.fully.paid) 


 0  1 
81 19 